In [51]:
import numpy as np
import pandas as pd
import random2
import os #os의 경우 기본적으로 주어지기 때문에 setup.py에 하지 않는다.

In [188]:
from SALib.analyze import sobol
from SALib.analyze import fast
from SALib.analyze import rbd_fast
from SALib.analyze import delta

## data

In [225]:
# change path to relative path - only for publishing
current_directory = os.path.dirname(os.path.abspath(__file__))
os.chdir(current_directory)

path = "./sampleData/concatenated_df.csv"
simul_data = pd.read_csv(path)

oPath = "./sampleData/"
O1 = sorted(np.loadtxt(oPath + "O1.txt"))
O2 = sorted(np.loadtxt(oPath + "O2.txt"))
O3 = sorted(np.loadtxt(oPath + "O3.txt"))



## simulation code

In [226]:
def simple_Simulation(x1: 'int', x2: 'int', x3: 'int', n = 10):
    '''
    to make simple simulation
    
    Parameters
    ----------
    x1 : parameter 1. range: 1 to 5
    x2 : parameter 2. range: 1 to 5
    x3 : parameter 3. range: 1 to 5
    n : the number of simulation runs

    Returns
    -------
    DataFrame
        A comma-separated values (csv) file is returned as two-dimensional
        data structure with labeled axes.

    Examples
    --------
    >>> simple_Simulation(x1 = 1, x2 = 3, x3 = 2, n = 11)
    '''
    
    global simul_data # globally declare
   
    # select data
    condition = (simul_data['x1'] == x1) & (simul_data['x2'] == x2) & (simul_data['x3'] == x3)
    filtered_df = simul_data[condition]
    
    dfs = []
    for i in range(n): # now, extracts by #n
        
        uniq_num = random2.choice(pd.unique(filtered_df['uniq_num']))
        chosen_df = filtered_df[filtered_df['uniq_num'] == uniq_num] #filter only uniq_num
    
        # now make new simulation data
        new_data = {
            'x1': [chosen_df['x1'].iloc[0]],
            'x2': [chosen_df['x2'].iloc[0]],
            'x3': [chosen_df['x3'].iloc[0]],
            'y1': [sorted(chosen_df['y1'].tolist())],
            'y2': [sorted(chosen_df['y2'].tolist())],
            'y3': [sorted(chosen_df['y3'].tolist())]
        }
        
        chosen_df = pd.DataFrame(new_data)

        dfs.append(chosen_df) # appended chosen_df
        
    result_df = pd.concat(dfs, axis=0, ignore_index=True) 
    
    # sort the list in the columns by ascending order
    def sort_list(lst):
        return sorted(lst)

    # apply 메서드를 사용하여 각 셀의 리스트들을 오름차순으로 정렬
    result_df['y1'] = result_df['y1'].apply(sort_list)
    result_df['y2'] = result_df['y2'].apply(sort_list)
    result_df['y3'] = result_df['y3'].apply(sort_list)

    
    return result_df

## 1) preprocessing (1) - Determine a criterions for calibration

In [228]:
# run multiple simulations

def multiple_simple_simulation(x1_list, x2_list, x3_list, M = 150, u = 0.1, k = 3):
    '''
    to make simple simulation results df by multiple parameters
    
    Parameters
    ----------
    x1: parameter 1. range: 1 to 5
    x2: parameter 2. range: 1 to 5
    x3: parameter 3. range: 1 to 5
    M: MonteCarlo index (default:100, too low:low accuracy, too high:computational intensity) 
    u = leniency index (default:0.1, too low:overfit, too high:uncertainty)
    k = the number of parameters (3)

    Returns
    -------
    DataFrame
        A comma-separated values (csv) file is returned as two-dimensional
        data structure with labeled axes.

    Examples
    --------
    >>> multi_simul_df = multiple_simple_simulation(x1_list, x2_list, x3_list, M = 150, u = 0.1, k = 3)
    '''    
    
    global simple_Simulation
    
    # list for saving all results dfs
    prep1_dfs = []
    
    for i in range(M*(2*k + 2)): #1200 times
        # set parameter space
        x_1 = random2.choice(x1_list)
        x_2 = random2.choice(x2_list)
        x_3 = random2.choice(x3_list)

        # run model and save
        tem_prep1_data = simple_Simulation(x1 = x_1, x2 = x_2, x3 = x_3, n = 1)

        # append temporal result to list
        prep1_dfs.append(tem_prep1_data)

    result_df = pd.concat(prep1_dfs, axis=0, ignore_index=True)

    return result_df

In [229]:
# Preprocessing (1): determining a criterion for calibration

#def prep1_criterion():

def prep1_criterion(O_list, multi_simul_df, u, k):
    '''
    As a preprocessing step, the root mean square error (RMSE) is calculated to determine the criterion for calibration.
    
    Parameters
    ----------
    O_list: list that includes observed data
    multi_simul_df: result of multiple simulation
    u: leniency index (default:0.1, too low:overfit, too high:uncertainty)
    k: the number of parameters (3)
    
    * If there are multiple y columns in multi_simul_df, they should be denoted as y1, y2, y3, y4, and so on.
    * Likewise, p column should be in the form of p1, p2, p3, p4, and so on.

    Returns
    -------
    DataFrame
        A comma-separated values (csv) file is returned as two-dimensional
        data structure with labeled axes.

    Examples
    --------
    >>> rmse_sel_df, multi_simul_df = prep1_criterion(O_list, multi_simul_df, u, k) 
    '''        
    
    multi_simul_df_temp = multi_simul_df.copy()
    
    # --- func for RMSE calculation ---
    def rmse(actual, predicted):
        return np.sqrt(np.mean((np.array(actual) - np.array(predicted))**2))


    # --- add combinations of y ---
    comb_columns = [col for col in multi_simul_df_temp.columns if col.startswith('x')] # if the comlumn name starts with x
    multi_simul_df_temp['comb'] = multi_simul_df_temp[comb_columns].apply(lambda row: list(row), axis=1)

    
    # --- add new columns of rmse between y columns and O_list ---
    for i, col in enumerate(multi_simul_df_temp.columns):
        if col.startswith('y'):
            col_name = 'rmse_O' + col[1:]
            # print(col[1:])
            multi_simul_df_temp[col_name] = multi_simul_df_temp[col].apply(lambda x: rmse(x, O_list[int(col[1:]) - 1]))
    
    # --- now, we need to calculate criterions for calibration for each y--- 
    # comb는 괜히 구함. 나중에 써먹기
    # 여기서는 rmse_O1, rmse_O2,... 등의 최소, 최대값을 구하고, rmse_sel_yn =  Y_j=Min(〖RMSE〗_tem )+(Max(〖RMSE〗_tem )-Min(〖RMSE〗_tem ))*μ  을 구하면 됌.
    
    # rmse_O 컬럼들 선택
    rmse_O_columns = [col for col in multi_simul_df_temp.columns if col.startswith('rmse_O')]

    # 각 rmse_O 컬럼들의 최소값과 최댓값 구하기
    min_values = multi_simul_df_temp[rmse_O_columns].min()
    max_values = multi_simul_df_temp[rmse_O_columns].max()

    # display(multi_simul_df_temp.head(2))
    
    # --- now, calculate RMSEsel for each y.
    # select rmse_O_ columns
    rmse_O_columns = [col for col in multi_simul_df_temp.columns if col.startswith('rmse_O')]

    # save the result by creating another df
    rmse_sel_df = pd.DataFrame()

    for col in rmse_O_columns:
        rmse_min = min_values[col]
        rmse_max = max_values[col]
        # print(col, rmse_min, rmse_max)
        # add the calculation result to new columns
        rmse_sel_df[col] = [rmse_min + (rmse_max - rmse_min) * u]
        rmse_sel = rmse_min + (rmse_max - rmse_min) * u
        
        # new columns for calculation
        multi_simul_df_temp[col + '_sel'] = rmse_sel
    
        

    return rmse_sel_df, multi_simul_df_temp
    
    


## 2) preprocessing (2) - Sorting Y and X

In [230]:
def sorting_Y(multi_simul_df_rmse_sel):
    '''
    Count the cases where 'rmse' is smaller than 'rmse_sel'. If the counts are higher, that 'y' is calibrated first.
    
    Parameters
    ----------
    multi_simul_df_rmse_sel: result of multiple simulation that includes rmse and rmse_sel
    
    Returns
    -------
    DataFrame
        A comma-separated values (csv) file is returned as two-dimensional
        data structure with labeled axes.

    Examples
    --------
    >>> y_seq_df = sorting_Y(multi_simul_df_rmse_sel)
    '''          
    
    # Columns that starts with rmse_O
    rmse_cols = [col for col in multi_simul_df_rmse_sel.columns if col.startswith('rmse_O')]
    num_rmse_cols = int(len(rmse_cols)/2)
    num_rmse_cols
    
    # Count rows that satisfies the condition (rmse < rmse_sel)
    result_df = pd.DataFrame()
    
    for i in range(1, num_rmse_cols + 1):
        rmse_col = f'rmse_O{i}'
        sel_col = f'rmse_O{i}_sel'
        count = multi_simul_df_rmse_sel[multi_simul_df_rmse_sel[rmse_col] < multi_simul_df_rmse_sel[sel_col]].shape[0]
        
        y_col = f'y{i}' # y_seq_df
        # y_seq_df = y_seq_df.append({'y': y_col, 'count': count}, ignore_index=True)

        y_col = f'y{i}'
        y_seq_df = pd.DataFrame({'y': [y_col], 'count': [count]})
        result_df = pd.concat([result_df, y_seq_df], ignore_index=True)
        
    # 'count' 컬럼을 기준으로 내림차순 정렬하여 'y' 값을 출력
    sorted_y_seq_df = result_df.sort_values(by='count', ascending=False)

    print('The order of Ys:', sorted_y_seq_df['y'].to_list())
    
    return result_df

In [231]:
def sorting_X(problem: dict, multi_simul_df_rmse_sel, GSA = 'RBD-FAST'):
    
    '''
    
    Sobol: Sobol’ Sensitivity Analysis
    FAST: Fourier Amplitude Sensitivity Test
    RBD-FAST: Random Balance Designs Fourier Amplitude Sensitivity Test
    Delta: Delta Moment-Independent Measure
    '''
    # x_array
    Xs = np.array(multi_simul_df_rmse_sel['comb'].to_list())
    
    # 'rmse_O'로 시작하고 '_sel'이 없는 컬럼들을 뽑아서 모든 값을 array로 만들어서 리스트에 저장
    rmse_o_columns = [col for col in multi_simul_df_rmse_sel.columns if col.startswith('rmse_O') and not col.endswith('_sel')]
    y_list = [np.array(multi_simul_df_rmse_sel[col]) for col in rmse_o_columns]


    Si_list = []

    for y in y_list:
        
        if GSA == 'Sobol':
            Si = sobol.analyze(problem, y)
            # print(Si['S1'])
        elif GSA == 'FAST':
            Si = fast.analyze(problem, y)
        elif GSA == 'RBD-FAST':
            Si = rbd_fast.analyze(problem, Xs, y)
        elif GSA == 'Delta':
            Si = delta.analyze(problem, Xs, y)
            
        Si_list.append(Si['S1']) # the first-order sensitivity indices
    
    # --- Now, we will return each first order sensitivity index
    # calculate average of sensitiviry indices
    averages = [sum(column) / len(column) for column in zip(*Si_list)]

    # new dataframe
    si_df = pd.DataFrame()

    # insert x1, x2, x2... into 'Xs' column
    si_df['Xs'] = [f'x{i}' for i in range(1, len(averages) + 1)]

    # calculate average of Si and put those to 'first_order_Si' column
    si_df['first_order_Si'] = averages
    
            
    # print 'x' by decending order based on 'count' column
    sorted_x_seq_df = si_df.sort_values(by='first_order_Si', ascending=False)

    print('The order of Xs:', sorted_x_seq_df['Xs'].to_list())
    
    
    return si_df

## 3) Parameter space searching and calibration

In [ ]:
# y_seq_df와 x_seq_df, multi_simul_df_rmse_sel를 이용해서 만들어야 함.
# multi_simul_df_rmse_sel를에서 x로 시작하는 컬럼. 뽑아서 각 x마다 unique한 값들을 뽑아내서 sort함.
# 이후 y_seq_df와 x_seq_df순서대로 calibration을 시작. y수만큼 진행됌.

# 사용자:

In [232]:
# 시뮬레이션을 무작위로 돌린다.

x1_list = [1,2,3,4,5]
x2_list = [1,2,3,4,5]
x3_list = [1,2,3,4,5]

# set hyper parameters
M = 150
u = 0.1
k = 3

# ---  run simulations for M(2k+2) times ---
multi_simul_df = multiple_simple_simulation(x1_list, x2_list, x3_list, M, u, k) 

In [234]:
# --- preprocessing 1: determining a criterion for calibration

O_list = [O1, O2, O3] # observed data to list -> sqp.O1, sqp.O2, sqp.O3 를 넣어야 함.
rmse_sel_df, multi_simul_df_rmse_sel = prep1_criterion(O_list, multi_simul_df, u, k)

# now, we have the rmse_sel for all O.
rmse_sel_df

,rmse_O1,rmse_O2,rmse_O3
0,425.547804,50.487752,3.477297


In [235]:
# --- preprocessing 2: sorting Y for calibration

y_seq_df = sorting_Y(multi_simul_df_rmse_sel)
y_seq_df

The order of Ys: ['y1', 'y3', 'y2']


,y,count
0,y1,595
1,y2,172
2,y3,330


In [236]:
# --- preprocessing 3: sorting X based on sensitivity analysis for calibration
problem = {
    'num_vars': 3,
    'names': ['p1', 'p2', 'p3'],
    'bounds': [[1, 5],
               [1, 5],
               [1, 5]]
}

x_seq_df = sorting_X(problem, multi_simul_df_rmse_sel, GSA = 'RBD-FAST')
x_seq_df

The order of Xs: ['x3', 'x2', 'x1']


,Xs,first_order_Si
0,x1,0.012919
1,x2,0.060078
2,x3,0.644702


In [237]:
multi_simul_df_rmse_sel

,x1,x2,x3,y1,y2,y3,comb,rmse_O1,rmse_O2,rmse_O3,rmse_O1_sel,rmse_O2_sel,rmse_O3_sel
0,2,5,2,"[62.7, 154.1, 218.0, 274.6, 302.0, 461.0, 651....","[20.0, 92.0, 110.0, 118.0, 155.0, 199.0, 226.0...","[13.0, 15.0, 15.0, 15.0, 16.0, 17.0, 18.0, 19....","[2, 5, 2]",1273.471428,278.839335,12.900581,425.547804,50.487752,3.477297
1,2,4,4,"[15.4, 29.7, 35.8, 49.1, 59.2, 63.9, 64.9, 67....","[9.0, 33.0, 33.0, 46.0, 51.0, 61.0, 65.0, 66.0...","[4.0, 7.0, 7.0, 7.0, 9.0, 10.0, 10.0, 11.0, 11...","[2, 4, 4]",104.789936,73.258788,4.221966,425.547804,50.487752,3.477297
2,4,3,3,"[14.5, 91.2, 101.7, 109.1, 140.5, 142.2, 143.3...","[20.0, 29.0, 95.0, 98.0, 98.0, 111.0, 119.0, 1...","[7.0, 11.0, 14.0, 14.0, 15.0, 15.0, 15.0, 16.0...","[4, 3, 3]",341.271789,109.885509,10.962436,425.547804,50.487752,3.477297
3,2,4,3,"[6.4, 38.0, 40.6, 70.4, 119.5, 154.3, 239.2, 2...","[8.0, 28.0, 36.0, 37.0, 82.0, 102.0, 137.0, 14...","[2.0, 7.0, 10.0, 11.0, 12.0, 12.0, 13.0, 14.0,...","[2, 4, 3]",320.498838,139.701915,7.464918,425.547804,50.487752,3.477297
4,2,2,1,"[56.7, 321.4, 336.9, 347.9, 471.4, 560.2, 689....","[41.0, 87.0, 87.0, 97.0, 97.0, 121.0, 136.0, 1...","[13.0, 13.0, 17.0, 17.0, 18.0, 19.0, 19.0, 20....","[2, 2, 1]",1749.861654,210.267984,13.578476,425.547804,50.487752,3.477297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,5,5,1,"[164.6, 328.0, 437.5, 461.3, 484.1, 485.5, 487...","[51.0, 118.0, 125.0, 134.0, 154.0, 201.0, 243....","[14.0, 17.0, 21.0, 22.0, 24.0, 25.0, 25.0, 25....","[5, 5, 1]",1347.487684,314.033119,18.405841,425.547804,50.487752,3.477297
1196,5,4,2,"[82.2, 98.2, 98.4, 140.0, 206.8, 231.9, 341.5,...","[30.0, 44.0, 45.0, 78.0, 81.0, 126.0, 164.0, 1...","[12.0, 17.0, 17.0, 19.0, 19.0, 20.0, 20.0, 21....","[5, 4, 2]",723.589090,173.031283,14.469796,425.547804,50.487752,3.477297
1197,5,5,3,"[36.8, 40.6, 44.6, 79.4, 101.6, 139.0, 145.5, ...","[29.0, 33.0, 61.0, 65.0, 88.0, 127.0, 131.0, 1...","[6.0, 8.0, 8.0, 10.0, 11.0, 11.0, 12.0, 13.0, ...","[5, 5, 3]",489.172839,156.245160,6.616268,425.547804,50.487752,3.477297
1198,5,5,5,"[16.8, 19.3, 19.5, 19.8, 22.1, 23.7, 24.9, 25....","[18.0, 18.0, 21.0, 23.0, 23.0, 25.0, 25.0, 28....","[3.0, 4.0, 5.0, 6.0, 7.0, 7.0, 7.0, 8.0, 8.0, ...","[5, 5, 5]",123.122328,52.944546,2.464752,425.547804,50.487752,3.477297
